In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import seaborn as sns

In [3]:
import sys

sys.path.append('../')
from classifiers import *

In [4]:
COLUMNS = [
    'segment_id',
    'split_index',
    'syn_density_shaft_after_proof',
    'spine_density_after_proof',
]

#### Load data

In [5]:
df_classifier = pd.read_pickle('../../data/data_tables/df_cell_types.pkl')
df_embeddings = pd.read_pickle('../../data/data_tables/df_classifier.pkl')
df_morphos = pd.read_pickle('../../data/data_tables/df_morphos.pkl')

In [6]:
df_classifier = pd.merge(df_classifier, df_embeddings, on=['segment_id', 'split_index'])
df_classifier = pd.merge(
    df_classifier, df_morphos[COLUMNS], on=['segment_id', 'split_index']
)
df_embeddings = pd.merge(
    df_embeddings, df_morphos[COLUMNS], on=['segment_id', 'split_index']
)

In [7]:
# Exclude fragmented neurons.
df_classifier = df_classifier[df_classifier['cell_type_coarse'] != 'ri']
df = df_embeddings[df_embeddings['ri_prediction'] == 'neuron']

# Exclude neurons without synaptic information.
df = df.dropna(subset=['syn_density_shaft_after_proof', 'spine_density_after_proof'])

In [8]:
np.unique(df_classifier['cell_type_coarse'], return_counts=True)

(array(['exc', 'inh'], dtype=object), array([922,  89]))

In [9]:
len(df)

37135

#### Fit classifier on labeled subset and predict for whole dataset

In [10]:
df = run_ei_classifier(df_classifier, df, 'data/')

Run cross-validation for ei-classifier.


100%|██████████| 7/7 [00:06<00:00,  1.14it/s]


Fit best model.
Run prediction for all neurons.
Prediction stats: (array(['exc', 'inh'], dtype='<U3'), array([33997,  3138])).
Model class: <class 'sklearn.linear_model._logistic.LogisticRegression'>
 Accurcay: 0.99
 Parameters: {'C': 5.0, 'class_weight': 'balanced', 'penalty': 'l2', 'l1_ratio': 0.5}

